In [1]:
import re
from pypdf import PdfReader
import pandas as pd

In [2]:
reader = PdfReader("Ed_30_PAS_3_2020_2022_Res_Final_Tipo D_Redação.pdf")
number_of_pages = len(reader.pages)

In [3]:
number_of_pages

297

In [4]:
text_list = []
for i in range(number_of_pages):
    text_list.append(reader.pages[i].extract_text())

In [5]:
len(text_list)

297

In [6]:
text = ''.join(text_list)

In [7]:
text

'1 \n UNIVERSIDADE DE BRASÍLIA (UnB)  \nCENTRO BRASILEIRO DE PESQUISA EM AVALIAÇÃO E  \nSELEÇÃO E DE PROMOÇÃO DE EVENTOS (CEBRASPE)  \nPROGRAMA DE AVALIAÇÃO SERIADA (PAS)  \nSUBPROGRAMA 2020 (TRIÊNIO 2020/2022)  \nEDITAL Nº 30 – PAS/UnB – SUBPROGRAMA 20 20, DE 24 DE JANEIRO DE 2023  \n \nA Universidade de Brasília (UnB) torna públicos o resultado final nos itens do tipo D , o resultado \nfinal na prova de redação em Língua Portuguesa  e o resultado final dos candidatos não eliminados na \nterceira etapa , referentes ao Programa de Avalia ção Seriada (PAS) – Subprograma 2020 (triênio \n2020/2022).  \n1 DO RESULTADO FINAL NOS ITENS DO TIPO D E DO RESULTADO FINAL NA PROVA DE REDAÇÃO EM \nLÍNGUA PORTUGUESA  \n1.1 Resultado final nos itens do tipo D e resultado final na prova de redação em Língua Portugu esa, na \nseguinte ordem: número de inscrição, nome do candidato em ordem alfabética, nota final nos itens do \ntipo D e nota final na prova de redação em Língua Portuguesa.  \n20195722, Aa

In [8]:
len(text)

1572415

In [9]:
text = re.sub(r" [0-9]+ \n", "", text)
text = re.sub(r"\n", "", text)
text = re.sub(r"^.*?(2\.1\.1 *)", r"\1", text)
text = re.sub(r"\*[^*]*$", "", text)

In [10]:
text

'2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)  20199023, Amanda Amorim Luz, 5.172, 14.653, 6.947, 3.845, 19.994, 7.222, 4.998, 16.660, 8.060, -25.699, 78, -, -, -, -, -, -, -, -, - / 20180980, Amanda Larissa Oliveira dos Santos, 6.896, 48.271, 3.800, 6.152, 54.342, 7.200, 5.712, 46.648, 6.924, 98.219, 1, -, -, -, -, -, -, -, -, - / 20188220, Amanda Luisa de Oliveira Souza, 0.000, 23.704, 5.333, 0.769, 24.303, 6.909, 2.142, 25.465, 8.148, -5.701, 57, -, -, -, -, -, -, -, -, - / 20193280, Amanda Mendes Reis de Araujo, 0.000, 10.342, 8.963, 2.307, 14.866, 9.5 86, 1.428, 13.088, 8.800, -44.840, 106, -, -, -, -, -, -, -, -, - / 20188170, Ana Beatriz Cattermol Cavalcante, 0.000, 10.917, 2.848, 0.000, 10.764, 3.750, 6.426, 17.373, 6.819, -49.856, 118, 1, -, -, -, -, -, -, -, - / 20107775, Ana Beatriz Gramkow Rangel , 3.448, 27.152, 5.071, 0.769, 16.660, 9.167, 1.428, 34.034, 8.575, 5.345, 40, -, -, -, -, -, -, -, -, - / 20183676, Ana Clara Marques da Silva, 0.862, 21.262,

In [11]:
# cutting the end
text = re.sub('\. \* Cursos que exigem.*', "", text)

In [12]:
string_sample = '2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)  20199023, Amanda Amorim Luz, 5.172, 14.653, 6.947, 3.845, 19.994, 7.222, 4.998, 16.660, 8.060, 2.1.3 CAMPUS  UNB CEILÂNDIA (FCE)  ENFERMAGEM (BACHARELADO)  20186508, Adrielle Moreira de Carvalho Lopes, 2.586, 17.526, 7.000, 2.307, 20.249, 5.412, 2.142,'

In [13]:
# split by course
pattern = r'\. (?=[A-Z]|\d\.)'
text_split = re.split(pattern=pattern, string=text)
#sections = [section for section in text_split if section.strip()]

In [14]:
len(text_split)

121

In [15]:
# weirdly some numeros de inscricao were broken into 6 digits + blank space + 2 digits, so we unite now
pattern = r"(\d{6})\s+(\d{2})"
sections = [re.sub(pattern, r"\1\2", section.strip()) for section in text_split]

In [16]:
def get_course_grades(my_string: str) -> tuple:
    
    pattern = "^(.*?)\d{8}, "

    result = re.search(pattern, my_string)
    if result:
        extracted_text = result.group(1)
        print(extracted_text)
    
    #course_dict = {}
    students = re.sub(re.escape(extracted_text), "", my_string)
    students = re.split(pattern=' /', string=students)
    
    #course_dict[extracted_text] = students

    return (extracted_text, students)

In [17]:
def convert_to_dataframe(my_tuple: tuple) -> pd.DataFrame:
    df = pd.Series(my_tuple[1])
    df = df.str.split(',', expand=True)
    df['course'] = my_tuple[0]
    return df

In [18]:
len(sections)

121

In [19]:
df_course = list()

for i in range(len(sections)):
    
    course = get_course_grades(sections[i])
    course = convert_to_dataframe(course)

    df_course.append(course)
    print(len(df_course))

2.1.1 CAMPUS  DARCY RIBEIRO – DIURNO  ADMINISTRAÇÃO (BACHARELADO)  
1
ADMINISTRAÇÃO (BACHARELADO) – SUB JUDICE   
2
AGRONO MIA (BACHARELADO)  
3
AGRONOMIA (BACHARELADO) – SUB JUDICE    
4
ARQUITETURA E URBANISMO (BACHARELADO)  
5
ARTES CÊNICAS - INTERPRETAÇÃO TEATRAL (BACHARELADO) *  
6
ARTES VISUAIS (BACHARELADO)*  
7
ARTES VISUAIS (LICENCIATURA)*    
8
BIBLIOTECONOMIA (BACHARELADO)  
9
BIOTECNOLOGIA (BACHARELADO)  
10
CIÊNCIA DA COMPUTAÇÃO (BACHARELADO)  
11
CIÊNC IA DA COMPUTAÇÃO (BACHARELADO) – SUB JUDICE    
12
CIÊNCIA POLÍTICA (BACHARELADO)  
13
CIÊNCIAS BIOLÓGICAS (BACHARELADO)  
14
CIÊNCIAS CONTÁBEIS (BACHARELADO)  
15
CIÊNCIAS ECONÔMICAS (BACHARELADO)  
16
CIÊNCIAS SOCIAIS – ANTROPOLOGIA / SOCIOLOGIA (BACHARELADO/LICENCIATURA)  
17
COMUNICAÇÃO SOCIAL – AUDIOVISUAL (BACHARELADO)  
18
COMUNICAÇÃO SOCI AL – AUDIOVISUAL (BACHARELADO) – SUB JUDICE    
19
COMUNICAÇÃO SOCIAL - PUBLICIDADE E PROPAGANDA (BACHARELADO)  
20
DESIGN – PROGRAMAÇÃO VISUAL/PROJETO DO PRODUTO (BACHARELADO)  
2

In [ ]:
pattern = "^(.*?)\d{8}, "

result = re.search(pattern, my_string)
if result:
    extracted_text = result.group(1)
    print(extracted_text)

#course_dict = {}
students = re.sub(re.escape(extracted_text), "", my_string)
students = re.split(pattern=' /', string=students)


In [20]:
teste = pd.concat(df_course)

In [24]:
teste.shape

(7300, 23)

In [36]:
COLUMN_NAMES = ['numero_inscricao',
                'nome',
                'escore_bruto_p1_etapa1',
                'escore_bruto_p2_etapa1',
                'nota_redacao_etapa1',
                'escore_bruto_p1_etapa2',
                'escore_bruto_p2_etapa2',
                'nota_redacao_etapa2',
                'escore_bruto_p1_etapa3',
                'escore_bruto_p2_etapa3',
                'nota_redacao_etapa3',
                'argumento_final',
                'classificacao_final_universal',
                'classificacao_final_cotas_negros',
                'classificacao_final_publicas1',
                'classificacao_final_publicas2',
                'classificacao_final_publicas3',
                'classificacao_final_publicas4',
                'classificacao_final_publicas5',
                'classificacao_final_publicas6',
                'classificacao_final_publicas7',
                'classificacao_final_publicas8',
                'course']

In [37]:
teste.columns = COLUMN_NAMES

In [39]:
teste.tail()

,numero_inscricao,nome,escore_bruto_p1_etapa1,escore_bruto_p2_etapa1,nota_redacao_etapa1,escore_bruto_p1_etapa2,escore_bruto_p2_etapa2,nota_redacao_etapa2,escore_bruto_p1_etapa3,escore_bruto_p2_etapa3,...,classificacao_final_cotas_negros,classificacao_final_publicas1,classificacao_final_publicas2,classificacao_final_publicas3,classificacao_final_publicas4,classificacao_final_publicas5,classificacao_final_publicas6,classificacao_final_publicas7,classificacao_final_publicas8,course
2,21270449,Sarah de Oliveira Nascimento,0.000,0.000,0.000,3.845,23.581,5.207,2.142,7.376,...,-,-,-,-,-,-,-,-,-,2.1.6 CAMPUS UNB PLANALTINA (FUP) – NOTURNO ...
0,20102521,Eloisa Graziele Rodrigues de Ara ujo,2.586,20.973,6.367,4.614,24.095,3.880,0.238,27.369,...,-,-,-,-,-,-,-,-,-,GESTÃO AMBIENTAL (BACHARELADO)
1,20190553,Julia de Sousa Vale,4.310,41.663,9.538,1.538,25.472,9.481,4.284,33.082,...,-,-,-,-,-,-,-,-,-,GESTÃO AMBIENTAL (BACHARELADO)
2,21208412,Manuella Stephanye Silva Magalhaes,0.000,0.000,0.000,3.845,35.068,7.308,3.570,24.752,...,-,-,-,-,-,-,-,1,-,GESTÃO AMBIENTAL (BACHARELADO)
3,20198986,Mateus Filipi Correia Silva,0.287,11.491,5.630,4.870,16.500,4.413,0.714,17.610,...,-,-,-,-,-,-,-,-,-,GESTÃO AMBIENTAL (BACHARELADO)
